In [1]:
# Select the review that bests represents a given topic
# Select the business with the highest proportion of a given topic

import cPickle as pickle
from gensim import corpora, models, similarities, utils
import numpy as np
import pandas as pd
from KAsql import query_SQL
import pyLDAvis.gensim
import pyLDAvis

In [2]:
lda = pickle.load(open('lda_ortho_30t_pos.p','rb'))
lda.print_topics()

[u'0.012*time + 0.008*well + 0.006*able + 0.006*staff + 0.006*great + 0.006*first + 0.005*years + 0.005*day + 0.005*knee + 0.005*took',
 u'0.012*knee + 0.007*going + 0.007*right + 0.007*office + 0.005*time + 0.005*need + 0.005*feel + 0.005*didnt + 0.005*years + 0.005*many',
 u'0.011*staff + 0.008*appointment + 0.007*know + 0.007*recommend + 0.007*first + 0.007*office + 0.006*minutes + 0.006*dont + 0.005*patient + 0.005*told',
 u'0.013*knee + 0.010*time + 0.009*staff + 0.008*office + 0.008*highly + 0.008*recommend + 0.007*experience + 0.005*first + 0.005*friendly + 0.005*care',
 u'0.017*office + 0.016*time + 0.008*told + 0.008*appointment + 0.007*even + 0.007*staff + 0.006*dont + 0.006*care + 0.006*said + 0.006*really',
 u'0.011*told + 0.009*said + 0.009*appointment + 0.009*time + 0.007*called + 0.007*insurance + 0.007*office + 0.007*mri + 0.006*didnt + 0.006*really',
 u'0.010*time + 0.008*mri + 0.007*insurance + 0.006*injury + 0.005*need + 0.005*never + 0.005*going + 0.004*even + 0.004

In [3]:
# Load dfs with gammas for all reviews and averaged by business/doctor
# All reviews

bid_df = pd.read_pickle('bid_tmeans.p')#index of this df is the bid, but can't be indexed by 'BID'
Reviews_topics_df = pd.read_pickle('Review_topics_df.p')

bid_topic_word_df = pd.read_pickle('bid_topic_word_df.p')
topic_word_df = pd.read_pickle('topic_word_df.p')
names_df = pd.read_pickle('names_df.p')

In [4]:
def three_representative_reviews(mytopic,Reviews_topics_df):
    """What 3 reviews best represent a given topic?
    Those with the highest topic-weights on one topic
    Revise to print top 3 reviews and see if they are consistent and validate with stars
    - use these reviews to help define & validate each topic"""

    # Sort on a given topic
    mysorted = Reviews_topics_df.sort(mytopic, axis=0, ascending=False)
    idx = mysorted.index[0:3]# get index of top-ranking reviews for mytopic (integer)

    RID = mysorted.loc[idx]['RID']
    RIDs = [str(int(value)) for value in RID.values]#list of strings (use int to get rid of decimal)

    #For debugging
    #print 'Topic: ',mytopic,'\n'
    #print 'RID: ',RID.values,'\n'#type Series
    
    # Pull 3 reviews, and see if they appear to represent one topic
    # SQL query uses KAsql2 function
    sql = 'SELECT stars, comment FROM review WHERE id='+RIDs[0]+' or id='+RIDs[1]+' or id='+RIDs[2]+';'
    rows = query_SQL(sql)

    stars=[];comment=''
    row0 = rows[:][0]
    row1 = rows[:][1]
    row2 = rows[:][2]
    
    rstars = row0[0],row1[0],row2[0]
    texts = row0[1],row1[1],row2[1]
    
    # Print Statements
    #print '\nTop three reviews most related to topic: ',mytopic,'\n'
    #print mysorted.head(3),'\n'#Confirm RIDs match with sql data

    # For some reason rows are printed in opposite order from listing
#     texts=[]
#     for i,row in enumerate(rows):
#         texts.append([row[0],row[1]])
    
    #print row[2][:500]
    return RID.values,rstars,texts
    

In [221]:
#print Reviews_topics_df.head()#RID and BID are first two columns then topic weights

#print bid_df.ix[309,:]

#print bid_topic_word_df.BID
#print Reviews_topics_df[Reviews_topics_df.BID==480]
#print bid_topic_word_df

# top = bid_topic_word_df.Topic[bid_topic_word_df.BID==480].values[0]
# word = bid_topic_word_df.Word[bid_topic_word_df.BID==480].values[0]

#print top,word

In [229]:
def save_txt(filename, text):
    with open(filename, "w") as text_file:
    #with open("Topic17_RR1.txt", "w") as text_file:
        text_file.write(text)

In [5]:
# Find the three most representative reviews for a given topic

Represent_Revs=pd.DataFrame()
series_list=[]
path = '/Users/kaschbacher/insightproject/yelp/git-yelp/doctopics/'

# Loop over BIDs
for each_id in bid_topic_word_df.BID:
    #print '\nBID: ',each_id
    temp_df = Reviews_topics_df[Reviews_topics_df.BID==each_id]#a subset of reviews for this BID
    temp_topic = bid_topic_word_df.Topic[bid_topic_word_df.BID==each_id]#extracts a series    
    #print temp_topic.values[0]#extract the topic as an int value from the series
    RIDS, rstars, texts= three_representative_reviews(temp_topic.values[0],temp_df)
    
    # Get the topic and word for this BID
    top = bid_topic_word_df.Topic[bid_topic_word_df.BID==each_id].values[0]
    word = bid_topic_word_df.Word[bid_topic_word_df.BID==each_id].values[0]
    
    #Does text contain the key-word?
    #print 'topic',top,' word',word
    for i, text in enumerate(texts):
        outcome = text.find(word)
        #print 'text_index:', outcome, RIDS[i], rstars[i]
#         if RIDS[i] in [23137,22088,23362]:
#             #pass
#             print text
        if outcome>-1:
        #if outcome>-1 and word not in ['hip']:#word was found in review
            print each_id, RIDS[i], rstars[i], 'top',top, word
            print text
            rid_series = pd.Series([each_id, RIDS[i], rstars[i], top, word])
            series_list.append(rid_series)
            filename = 'BID'+str(each_id)+'_Topic'+str(top)+'_RID'+str(RIDS[i])+'.txt'
            print path+filename,'\n'
            #save_txt(path+filename, text)

Represent_Revs = pd.concat(series_list, axis=1).transpose()
Represent_Revs.columns = ['BID',"RID",'rstars','Topic','Word']

# Adding a couple hand-selected physicians
#Dickenson
Represent_Revs = Represent_Revs.append({'BID':319,'RID':20802,'rstars':4,'Topic':22,'Word':'hip'},ignore_index=True)
#Akizuki
Represent_Revs = Represent_Revs.append({'BID':309,'RID':20609,'rstars':4,'Topic':17,'Word':'knee'},ignore_index=True)
print Represent_Revs
#print new_df


USE ortho;
USE ortho;
353 21350 5.0 temptop 0 top 0 hip
I need a new hip and my primary care doctor told me if he was to have this surgery he would have Doctor Gilbert do it. I went there and was so surprised to have a doctor who spent over an hour with me and seemed concerned about me.  I am so confident that after this surgery the end of December I will once again be able to walk without the current bone on bone pain which I am experiencing. The staff was so friendly and all the magazines in the waiting room were new. He also has called me twice to see how I am coping.
/Users/kaschbacher/insightproject/yelp/git-yelp/doctopics/BID353_Topic0_RID21350.txt 

353 21347 5.0 temptop 0 top 0 hip
Had a left hip revision and he was amazing. We went over the many options and I wanted to be aggressive and take out my 17 year old hip and put in one of the newer models. He convinced me otherwise which it turns out it was the right thing to do because my hip was so grown in it would have destroyed 

In [239]:
Represent_Revs.to_pickle('Represent_Revs.p')
Represent_Revs.to_pickle('doctopics/Represent_Revs.p')

In [7]:
# Work around because AWS pandas version is outdated and can't read pickles
# create dict, and reads the dict - converting back to DF in views.py
rr = Represent_Revs.to_dict()
rr_pickled = pickle.dumps(rr)
f = open('rr_pickled', 'w')
f.write(repr(rr_pickled))
f.close()

In [215]:
print Represent_Revs.RID[Represent_Revs.BID==309].values[0]

print bid_topic_word_df.Topic[bid_topic_word_df.BID==588].values[0]

20609
11


In [238]:
myrev1 = "BID566_Topic16_RID23137.txt'"
path = "'static/img/"
print path+myrev1

'static/img/BID566_Topic16_RID23137.txt'


In [155]:
# with open("Topic22_RR2.txt", "r") as text_file:
#     lines = text_file.read()  #type string
# print type(lines)
# print lines


In [83]:
def most_representative_review(mytopic,Reviews_topics_df):
    """ What review best represents a given topic?
    Selects the review with the highest gamma-weights on one topic.
    Accepts mytopic (an int corresponding to topic #) and the Rev x G matrix with rid, bid and gammas"""

    #print(ids_gammas_df.head())#Check what df looks like prior to sorting

    # Sort on a given topic
    mysorted = Reviews_topics_df.sort(mytopic, axis=0, ascending=False)
    idx = mysorted.index[0]# get index of top-ranking review for mytopic

    RID = int(mysorted.loc[idx]['RID'])
    print 'Index = ',idx, ' Starting RID: ',RID,' Topic: ',mytopic,'\n'#Note that if you replace idx with 0, and print the results, it is not the same
    #print(mysorted.loc[0]['RID'])# does not work

    #from KAsql import query_SQL
    sql = 'SELECT id, stars, comment FROM review WHERE id='+str(RID)+';'
    rows = query_SQL(sql)

    sqlid = rows[0][0]
    stars = rows[0][1]
    comment = rows[0][2]
    
    #Print statements
    print sql,'\n'
    print '\nTop review most related to topic: ',mytopic,'\n'
    print(mysorted.head(3))
    print str(RID), sqlid, stars, comment